In [ ]:
import intake
import xarray as xr

df = intake.open_csv("./data/bird_migration/{species}.csv").read()
air = xr.open_dataset("./data/air.nc").air

## Interactive Pipelines

In the previous notebooks we used `hvplot` to create interactive visuals be passing in columns as axes or color and sometimes using groupby. Sometimes though you want to interactively slice up your data along other dimensions. 

In [ ]:
import hvplot.pandas
import hvplot.xarray
import panel as pn

pn.extension(sizing_mode="stretch_width")

To get started we will create some `panel` widgets. You can see a list of all the widget options in the [Widget Reference Gallery](https://panel.holoviz.org/reference/index.html#widgets).

In [ ]:
day_slider = pn.widgets.IntSlider(name="Day of the Year", start=1, end=365)
day_slider

In [ ]:
day_slider.value

Now with our widget in hand, we can create an interactive dataset.

In [ ]:
air_i = air.interactive()

air_i.sel(day=day_slider)

We can use that interactice dataset as the basis for other visuals

In [ ]:
air_i.sel(day=day_slider).hvplot(geo=True, coastline=True)

We can put these two views together by putting them in a `panel.Row`.

In [ ]:
day_slider = pn.widgets.IntSlider(name="Day of the Year", start=1, end=365)

pn.Row(
    air_i.sel(day=day_slider),
    air_i.sel(day=day_slider).hvplot(geo=True).panel(margin=(30, 30)),
)

Now let's see how we can slice up the bird data. First we can make a species selector widget.

In [ ]:
species_selector = pn.widgets.Select(
    name="Bird Species", options=list(df.species.cat.categories)
)
species_selector

Now let's see who is around that bird at any given day of the year. We'll start by getting the lat and lon for the bird that we've selected.

In [ ]:
df_i = df.interactive()

lon, lat = df_i[(df_i.species == species_selector) & (df_i.day == day_slider)][
    ["lon", "lat"]
].values[0]
lon

Now we can use that lon and lat as input to another selector

In [ ]:
df_i[
    (df_i.day == day_slider)
    & (df_i.lon > lon - 5)
    & (df_i.lon < lon + 5)
    & (df_i.lat > lat - 5)
    & (df_i.lat < lat + 5)
].set_index(["lon", "lat"]).sort_index()["species"]

For a longer discussion of `.interactive` check out the [HoloViz Tutorial](https://holoviz.org/tutorial/Interactive_Pipelines.html) or the [HoloViz Docs](https://hvplot.holoviz.org/user_guide/Interactive.html)

[Next Section](./04_panel.ipynb)